# Problème n°2: PointNet

Certains jeux de données impliquent des nuages de points dans un espace 3D. Penser par exemple à un ensemble de mesures lidar : chaque tir permet de renseigner les coordonnées d'un des points de l'objet ciblé.
Une tâche intéressante consiste à classer chacun des points du nuage en fonction de l'objet auquel il appartient. Cette tâche est considérée comme une variante de la segmentation sémantique d'images.

Ce problème introduit à une méthode directe de segmentation d'un nuage par deep learning. Elle est basée sur une architecture particulière appelée PointNet. \
Dans la première partie, on présente un jeu de données (synthétisé à la volée) impliquant des nuages de points.
Dans la seconde partie, on explore la structure et les propriétés de PointNet. Dans la troisième, on l'entraîne et dans la dernière partie, on charge les poids d'une version améliorée de PointNet (PointNet++) pour comparaison.

La cellule suivante permet les imports nécessaires:

In [4]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import matplotlib.pyplot as plt
import os
! pip install einops
! git clone https://github.com/nouhalahyen/exam_2025.git
! cp exam_2025/utils/utils_probleme2.py .
from utils_probleme2 import gen_pointcloud, plot_triplets

RuntimeError: function 'conv1d' already has a docstring

## Partie I : un exemple de PointCLoud data

Pour construire le jeu de données, on simule un terrain couvert de deux types de bâtiments : des immeubles de forme rectangulaire aux toits plats et des igloos (dômes). Pour créer les nuages, on échantillonne les surfaces vues du ciel (les murs des bâtiments rectangulaires ne sont pas échantillonnées), en favorisant les zones d'altitude non nulles.
Le but est de distinguer les igloos du reste (sol et toits des bâtiments). Il s'agit donc d'une segmentation binaire.

In [3]:
batch_size = 6
input_points, target_list, target_points  = gen_pointcloud(batch_size)


for i in range(batch_size):
  print(i)
  # Représentation 3D des nuages de points et
  # les paramètres elev et azim permettent de changer l'angle de vue
  plot_triplets(input_points[i].transpose(0,1).cpu(),
                elev=75, azim=0)

  # Cibles : les points appartenant aux toitures d'igloos sont
  # dans la classe 1, les autres, dans la classe 0.
  plot_triplets(target_points[i].transpose(0,1).cpu(),
                title='Cibles',
                cbar_label='classe')

  # Note: target_points contient non seulement les classes
  # mais aussi les coordonnées x et y des points, pour
  # faciliter leur visualisation

NameError: name 'gen_pointcloud' is not defined

**Q1** A quoi correspondent les différentes dimensions de *input_points* ?

N x 3 : Où N est le nombre de points dans le nuage, et chaque point est défini par ses coordonnées (x, y, z) en 3D.
Chaque ligne représente un point unique du nuage, et les colonnes contiennent les informations suivantes :

    x : Coordonnée spatiale horizontale (axe 1).
    y : Coordonnée spatiale verticale (axe 2).
    z : Hauteur ou altitude du point.

**Q2** Les points d'un nuage sont-ils rangés dans un ordre particulier ?

Les coordonnées en entrée de input_points servent à :

    Décrire la structure géométrique du nuage de points en 3D.
    Permettre la segmentation binaire :
        Les coordonnées (x, y, z) aident à différencier les formes (plat pour les immeubles et courbé pour les igloos).
    Capturer les caractéristiques spécifiques à chaque type de bâtiment (ex. : hauteur constante pour les toits plats vs variation pour les dômes).

**Q3** (question ouverte). Si vous deviez traiter le problème avec un FCN ou un ViT (Visual Transformer), que feriez-vous ?

1. Traitement avec un FCN

Un FCN est conçu pour des données structurées comme des images 2D ou des volumes 3D réguliers. Voici comment je procéderais :
Étapes :

    Représentation des nuages de points :
        Convertir les nuages de points 3D en une grille régulière (voxelisation). Chaque voxel serait une cellule 3D contenant un point ou un ensemble de points.
        Les valeurs dans la grille pourraient être des indicateurs binaire (présence/absence d’un point) ou des statistiques locales (ex. : densité, coordonnées moyennes).

    Entrée du modèle :
        Alimenter cette grille voxelisée dans un FCN 3D qui applique des convolutions volumétriques pour extraire les caractéristiques locales et globales.

    Prédictions :
        Produire une segmentation volumétrique où chaque voxel est classifié comme appartenant ou non à un igloo.

Avantages :

    Exploite directement les capacités des FCNs pour traiter des données structurées.
    Approche bien comprise et optimisée pour les grilles régulières.

Inconvénients :

    Perte d’information due à la voxelisation, en particulier si la grille est de faible résolution pour limiter la mémoire.
    Consommation mémoire élevée, car une grille volumétrique dense peut être très coûteuse.

2. Traitement avec un ViT

Les ViTs peuvent capturer les relations globales entre les points, mais leur utilisation sur des nuages de points nécessite une adaptation importante.
Étapes :

    Extraction de patchs :
        Traiter les points 3D comme une séquence d'entités. Chaque point ou petit groupe de points serait transformé en un vecteur d'entrée via une petite MLP (Multi-Layer Perceptron) ou un encodage géométrique (ex. : coordonnées (x, y, z) avec des features comme la densité locale).

    Encodage positionnel :
        Ajouter des informations de position (comme dans les ViTs pour les images) pour indiquer la localisation spatiale des points.

    Application du Transformer :
        Passer la séquence de vecteurs à travers un Transformer classique (avec des mécanismes d'attention) pour capturer les relations globales entre les points.

    Prédictions finales :
        Utiliser une tête de segmentation pour classer chaque point en fonction de ses caractéristiques globales et locales apprises par le Transformer.

Avantages :

    Capture les relations globales entre les points grâce au mécanisme d'attention.
    Flexible pour des tâches de segmentation et classification.

Inconvénients :

    Gourmand en données : Les ViTs nécessitent généralement un grand volume de données annotées pour apprendre efficacement.
    Coût computationnel élevé, surtout pour des séquences longues (grands nuages de points).



## Partie II : le modèle PointNet

Dans cette partie, on s'intéresse à la propriété principale d'un réseau PointNet : l'utilisation d'opérations invariantes par rapport à l'ordre dans lequel les points sont présentés au réseau.

In [5]:
from utils_probleme2 import PointNetSegHead
pointnet = PointNetSegHead(num_points=800, num_global_feats=1024, m=2).cuda()

input_points, target_list, _ = gen_pointcloud(batch_size)
input_points = input_points.cuda()
output, _, _ = pointnet(input_points)

ModuleNotFoundError: No module named 'utils_probleme2'

**Q1** La sortie du modèle PointNet correspond au premier tenseur du *tuple* fourni la fonction *forward* de *pointnet*. A quoi correspondent les différentes dimensions de *output* ? Quel est l'effet d'une permutation des points contenus dans *inputs_points* sur la sortie ? Répondre :

- en vous référant à l'article [l'article](https://arxiv.org/abs/1612.00593) qui introduit ce réseau (citer dans le texte).
- à partir de tests à effectuer dans la cellule de code suivante (utiliser torch.randperm pour générer des permutations sur les entrées)

Dimensions de output :

    Si l'entrée est de dimension (batch_size, N, D) où :
        batch_size : Nombre de nuages de points dans un batch.
        N : Nombre de points dans chaque nuage.
        D : Dimensions des caractéristiques pour chaque point (e.g., coordonnées (x, y, z)).
    La sortie output de PointNet a typiquement deux parties :
        Un vecteur global (extrait par agrégation, par exemple via max pooling) de dimension (batch_size, G) où G est la dimension des caractéristiques globales.
        Des caractéristiques locales pour chaque point de dimension (batch_size, N, F) où F est la dimension des caractéristiques pour chaque point.

Effet d'une permutation :

    PointNet est conçu pour être invariant aux permutations des points grâce aux mécanismes suivants :
        Les opérations appliquées point par point (comme des MLP partagées) ne dépendent pas de l'ordre.
        L'agrégation globale (souvent via max pooling) combine les informations de manière indépendante de l'ordre.
    Ainsi, une permutation des points dans input_points ne modifie pas la sortie globale (vecteur agrégé).

Justification : Dans l'article PointNet (Qi et al., 2017), cette propriété est obtenue par l'utilisation de :

    Opérations partagées (les mêmes transformations pour chaque point).
    Agrégation symétrique (ex. : max pooling), garantissant que l’ordre n’impacte pas le résultat.

**Q2** L'architecture de *pointnet* est décrite dans la Figure 2 de l'article (voir ci-dessous) évoqué à la question précédente. En dehors des opérations notées "input transform" et "feature transform", dont la compréhension est plus délicate, quelles sont les différentes opérations conduisant à une segmentation ? Que signifie le terme "shared" et expliquer en quoi ces opérations sont invariantes par rapport à l'ordre de présentation des points.

<img src= https://miro.medium.com/v2/resize:fit:1100/format:webp/1*lFnrIqmV_47iRvQcY3dB7Q.png >

Opérations conduisant à la segmentation :

    MLP partagées : Des couches fully connected appliquées à chaque point indépendamment. Chaque point est traité de la même manière pour extraire des caractéristiques locales.
    Max pooling : Une opération d’agrégation symétrique qui extrait des caractéristiques globales du nuage en combinant les informations locales.
    Concaténation : Les caractéristiques locales (par point) et globales (vecteur agrégé) sont combinées pour produire une segmentation fine.
    MLP finales : Les couches fully connected finales effectuent la classification (binaire ou multi-classes) de chaque point.

Signification de "shared" :

    Les poids des MLP sont partagés pour tous les points d’un nuage.
    Cela garantit que chaque point est traité de manière identique, indépendamment de son positionnement ou de l'ordre des autres points.

Opérations conduisant à la segmentation :

    MLP partagées : Des couches fully connected appliquées à chaque point indépendamment. Chaque point est traité de la même manière pour extraire des caractéristiques locales.
    
    Max pooling : Une opération d’agrégation symétrique qui extrait des caractéristiques globales du nuage en combinant les informations locales.
    Concaténation : Les caractéristiques locales (par point) et globales (vecteur agrégé) sont combinées pour produire une segmentation fine.
    MLP finales : Les couches fully connected finales effectuent la classification (binaire ou multi-classes) de chaque point.

Signification de "shared" :

    Les poids des MLP sont partagés pour tous les points d’un nuage.
    Cela garantit que chaque point est traité de manière identique, indépendamment de son positionnement ou de l'ordre des autres points.

Les MLP appliquent la même transformation à chaque point, ce qui ne dépend pas de l’ordre.
L'agrégation symétrique (e.g., max pooling) garantit que l’ordre n’influence pas le vecteur global.

## Partie III

Dans cette partie, on se propose d'entraîner un PointNet. Pour ce faire, on utilisera une fonction de coût spécifique (voir cellule ci-dessous).

**Consignes :**

1) Entraîner un PointNet sur quelques centaines d'époques.

2) Afficher à chaque époque la justesse des prédictions

3) Charger les poids d'un réseau entraîné sur 500 époques, stockés dans le fichier **pointnet_500_ep.pth** du répertoire https://huggingface.co/nanopiero/pointnet_igloos.

Visualiser les sorties de ce modèle-là en complétant le la dernière cellule de code du calepin.


In [ ]:
optimizer = torch.optim.Adam(pointnet.parameters(),
                             lr=0.0001, betas=(0.9, 0.999))

# manually set alpha weights
alpha = np.array([0.2, 0.8])
gamma = 1
loss_fn = PointNetSegLoss(alpha=alpha, gamma=gamma, dice=True).cuda()

# exemple d'utilisation de PointNetSegLoss:
# La transposition permet de repasser la dimension relative
# aux probabilités en dernier, comme avec torch.nn.CrossEntropyLoss
proba_pred_list = outputs.transpose(1,2)
loss_fn(proba_pred_list, target_list)

In [ ]:
batch_size = 64
n_epochs = 200
n_batch_per_epoch = 10


for epoch in range(1, n_epochs):
  print('epoch : ', epoch)
  for batch in range(1,n_batch_per_epoch):
    ...

In [ ]:
input_points, target_list , target_points = gen_pointcloud(6)

# Il faut construire les prédictions.
proba_pred_list, _, _ = pointnet2.cuda()(input_points.to(device))
pred_list = proba_pred_list.transpose(1,2).max(1)[1].cpu()

# Accuracy:
...
# Tracé

for i in range(6):
  print(i)
  plot_triplets(input_points[i].transpose(0,1), elev=75, azim=0)
  plot_triplets(target_points[i].transpose(0,1),
                title='Cibles',
                cbar_label='classe')
  plot_triplets(...,
                title='Predictions',
                cbar_label='classe')
